# Basic Conception

This notebook follows SpikingJelly's [Basic Conception tutorial](https://spikingjelly.readthedocs.io/zh-cn/0.0.0.0.14/activation_based_en/basic_concept.html).

## Activation-based Representation

`spikingjelly.activation_based` uses tensors with 0s or 1s to represent spikes. In this example, a value is transformed into 0 or 1 depending on whether or not it reaches a predefined threshold value.

In [1]:
import torch

v = torch.rand((8))
v_th = 0.5
spike = (v >= v_th).to(v)
print("Spike = ", spike)

Spike =  tensor([1., 1., 1., 1., 1., 0., 1., 0.])


## Step 

`spikingjelly.activation_based` supports two formats of data:

- **Single-step ("s"):** `shape = (N, *)` where `N` is the batch dimension and `*` is any extra dimensions
- **Multi-step ("s"):** `shape = (T, N, *)` where `T` is the time-step dimension



In [2]:
import torch
from spikingjelly.activation_based import neuron

net = neuron.IFNode(step_mode='m')
# 'm' is the multi-step mode
net.step_mode = 's'
# 's' is the single-step mode

         Only Prophesee DVS demo will not run properly.
         Please install it from https://github.com/prophesee-ai/prophesee-automotive-dataset-toolbox


To input a sequence data with `shape = (T, N, *)` to a single-step module, we must feed it as `T` tensors with shape `shape = (N, *)`

In [3]:
import torch
from spikingjelly.activation_based import neuron, functional

net_s = neuron.IFNode(step_mode='s')
T, N, C, H, W = 4, 1, 3, 8, 8

x_seq = torch.rand((T, N, C, H, W))

########
# This #
########
y_seq = []
for t in range(T):
    x = x_seq[t] # x.shape = (N, C, H, W)
    y = net_s(x) # y.shape = (N, C, H, W)
    y_seq.append(y.unsqueeze(0))

y_seq = torch.cat(y_seq) # y_seq.shape = (T, N, C, H, W)


#########################
# Is equivalent to this #
# #######################
y_seq = functional.multi_step_forward(x_seq=x_seq, single_step_module=net_s) # y_seq.shape = (T, N, C, H, W)


###################################################
# But it's easier to just use a multi-step module #
###################################################
net_m = neuron.IFNode(step_mode='m')
y_seq = net_m(x_seq) # y_seq.shape = (T, N, C, H, W)

## Hidden States

Neurons and other SNN modules have hidden states which, together with the input data, determine their outputs at particular time-step. We represent the hidden-state at timestep $t$ as $H(t)$

$$Y(t) = f(X(t), H(t-1))$$

As such, the state of the network should be reset between the processing of different batches

In [ ]:
import torch
from spikingjelly.activation_based import neuron, functional

def print_checkpoint(net_s: neuron.IFNode, checkpoint_name: str, x=None, y=None):
    s = f"- {checkpoint_name} - "
    if not x is None:
        s += f"\n    x = {x}"
    if not y is None:
        s += f"\n    y = {y};    "
    print(s + f"\n    v = {net_s.v}\n")

net_s = neuron.IFNode(step_mode='s')

print(net_s)
print_checkpoint(net_s, "Start")

x = torch.rand((4))
y = net_s(x)
print_checkpoint(net_s, "First pass", x, y)

functional.reset_net(net_s) # Resets all stateful modules in network. net_s.reset() only resets net_s
print_checkpoint(net_s, "Reset")

x = torch.rand(8)
y = net_s(x)
print_checkpoint(net_s, "Second pass", x, y)

IFNode(
  v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=s, backend=torch
  (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
)
- Start - 
    v = 0.0

- First pass - 
    x = tensor([0.0609, 0.4822, 0.7468, 0.6863])
    y = tensor([0., 0., 0., 0.]);    
    v = tensor([0.0609, 0.4822, 0.7468, 0.6863])

- Reset - 
    v = 0.0

- Second pass - 
    x = tensor([0.8614, 0.6856, 0.1082, 0.2517, 0.1276, 0.9547, 0.7902, 0.0775])
    y = tensor([0., 0., 0., 0., 0., 0., 0., 0.]);    
    v = tensor([0.8614, 0.6856, 0.1082, 0.2517, 0.1276, 0.9547, 0.7902, 0.0775])



## Propagation Patterns

Continue from here!